In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import pickle
import seaborn as sns
from matplotlib import pyplot as plt

from simpleredcal.align_utils import align_df
from simpleredcal.plot_utils import arr_pcmesh, df_heatmap
from simpleredcal.red_likelihood import decomposeCArray, degVis, \
makeCArray, red_ant_sep
from simpleredcal.red_utils import find_deg_df, find_rel_df, RESPATH

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib as mpl

plot_figs = False
if plot_figs:
    mpl.rcParams['figure.dpi'] = 300

mpl.rc('font',**{'family':'serif','serif':['cm']})
mpl.rc('text', usetex=True)
mpl.rc('text.latex', preamble=r'\usepackage{amssymb} \usepackage{amsmath}')

In [ ]:
jd_time = 2458098.43869
jd_anchor = 2458099
pol = 'ee'
ndist = 'gaussian'

rel_dir_path = os.path.join(RESPATH, 'rel_dfs')
deg_dir_path = os.path.join(RESPATH, 'deg_dfs')

# Comparing a pair of datasets

## Loading 1st relatively calibrated dataframe

In [ ]:
with open(os.path.join(rel_dir_path, 'rel_df.{}.{}.md.pkl'.format(jd_time, pol)), \
          'rb') as f:
    md = pickle.load(f)
    
indices = ['freq', 'time_int']
resid_cols = ['residual', 'norm_residual']
vis_list = list(map(str, np.arange(md['no_unq_bls']*2).tolist()))
cvis_list = ['C' + vis_id for vis_id in list(map(str, np.arange(md['no_unq_bls']).tolist()))]
gain_list = list(map(str, np.arange(md['no_unq_bls']*2, (md['no_unq_bls'] + md['no_ants'])*2 ).tolist()))

In [ ]:
rel_df_path = find_rel_df(jd_time, pol, ndist, rel_dir_path)
rel_df = pd.read_pickle(rel_df_path)
rel_df.drop(columns=resid_cols, inplace=True)

Nfreqs = rel_df.index.get_level_values('freq').unique().size
Ntints = rel_df.index.get_level_values('time_int').unique().size

## Loading 2nd relatively calibrated dataframe

Due to an offset in LAST, two relatively calibrated dataframes must be merged, with the appropriate cuts in LAST to align the merged dataframe with the 1st one

In [ ]:
rel_df_c = align_df('rel', jd_time, jd_anchor, rel_dir_path, ndist, pol)
rel_df_c.drop(columns=resid_cols+gain_list, inplace=True)

## Degenerate transformation of the 1st dataframe

In [ ]:
min_list = ['success', 'status', 'message', 'fun', 'nit']
rel_df_d = rel_df[min_list].copy()
rel_df_d = rel_df_d.reindex(columns=rel_df_d.columns.values.tolist() + vis_list)

In [ ]:
deg_df_path = find_deg_df(jd_time, pol, 'jd.{}'.format(jd_anchor), ndist, deg_dir_path)
deg_df = pd.read_pickle(deg_df_path)

In [ ]:
deg_df_d = deg_df[['0', '1', '2']].copy().reset_index()
deg_df_d.rename(columns={'time_int1': 'time_int', '0': 'amp', '1': 'tilt_x', '2':'tilt_y'}, inplace=True)
deg_df_d.set_index(indices, inplace=True)
deg_df_d.sort_index(inplace=True)
rel_df.drop(columns=gain_list, inplace=True)
rel_df = rel_df.join(deg_df_d)
rel_df.sample(5).sort_index()

In [ ]:
ant_sep = red_ant_sep(md['redg'], md['antpos'])
rel_df_d[vis_list] = rel_df.apply(lambda row: pd.Series(decomposeCArray(degVis(ant_sep, \
                     makeCArray(row[len(min_list):len(min_list) + md['no_unq_bls']*2].values.astype(float)), \
                     *row[-3:].values.astype(float)))), axis=1)

In [ ]:
rel_df_d.sample(5).sort_index()

## Combining degenerately consistent dataframes

In [ ]:
# merging dataframes
rel_df_d['JD'] = int(jd_time)
rel_df_c['JD'] = int(jd_anchor)

rel_df_t = pd.concat([rel_df_d, rel_df_c])

rel_df_t.reset_index(inplace=True)
rel_df_t.set_index(['freq', 'time_int', 'JD'], inplace=True)
rel_df_t.sort_index(inplace=True)

In [ ]:
rel_df_t.sample(5).sort_index()

## Statistics on combined dataframe

In [ ]:
rel_df_t[vis_list].groupby(level=['freq', 'time_int']).mean()

In [ ]:
rel_df_t[vis_list].groupby(level=['freq', 'time_int']).std()

## Plots

### Single time integration

In [ ]:
time_integration = 40

In [ ]:
# selecting time integration
rel_df_tint = rel_df_t.xs(time_integration, level='time_int', drop_level=True)
# turning into complex values
rel_df_tintc = rel_df_tint.apply(lambda row: pd.Series(makeCArray(row[vis_list].values.astype(float))), \
                                 axis=1)

In [ ]:
vis_abs_mean = rel_df_tintc.abs().groupby('freq').mean()
piv = pd.pivot_table(vis_abs_mean, columns='freq')
vmax = np.nanpercentile(piv.values, 95)

# mean visibility amplitudes for specified time integration
arr_pcmesh(piv.columns, piv.index, piv.to_numpy(), vmin=0, vmax=vmax, extend='max', 
           xlabel='Frequency Channel', ylabel='Redundant Baseline Group', clabel=r'$\overline{|V|}$', \
           xlim=(0, 1023))

In [ ]:
re_df = rel_df_tint.drop(columns=min_list).iloc[:, np.arange(2*md['no_unq_bls'], step=2)]\
        .groupby('freq').var()
im_df = rel_df_tint.drop(columns=min_list).iloc[:, np.arange(1, 2*md['no_unq_bls'], step=2)]\
        .groupby('freq').var()
re_df.columns = np.arange(md['no_unq_bls'])
im_df.columns = np.arange(md['no_unq_bls'])
var_df = re_df + im_df

In [ ]:
piv = pd.pivot_table(var_df, columns='freq')
vmax = np.nanpercentile(piv.values, 95)

# visibility variance for specified time integration
arr_pcmesh(piv.columns, piv.index, piv.to_numpy(), vmin=0, vmax=vmax, extend='max', 
           xlabel='Frequency Channel', ylabel='Redundant Baseline Group', clabel=r'$\mathrm{Var}(V)$', \
           xlim=(0, 1023), sci_fmt=True)

# Statistics over multiple JDs

Starting from the JDs from the previous section, we add further JDs that cover the same LAST range by aligning them in LAST and degenerately transforming them to be consistent with the anchor day (JD 2458099 in this case).

We present an example of how this is done below. See the align_deg.py script for the full implementation.

In [ ]:
# find dataset from specified JD that contains visibilities at the same LAST

jd_ci = 2458101

# load rel cal dataframe and align in LAST
rel_dfk = align_df('rel', jd_time, jd_ci, rel_dir_path, ndist, \
                   pol)
rel_dfk.drop(columns=resid_cols+gain_list, inplace=True)

# degenerate transformation
deg_dfk = align_df('deg', jd_time, jd_ci, deg_dir_path, ndist, \
                   pol, JD_anchor=jd_anchor)

# degenerate transformation of redundant visibility solutions
deg_dfk = deg_dfk[['0', '1', '2']].copy().reset_index()
deg_dfk.rename(columns={'time_int1': 'time_int', '0': 'amp', \
                        '1': 'tilt_x', '2':'tilt_y'}, inplace=True)
deg_dfk.set_index(indices, inplace=True)
deg_dfk.sort_index(inplace=True)
rel_dfk = rel_dfk.join(deg_dfk)

rel_df_di = rel_df[min_list].copy()
rel_df_di = rel_df_di.reindex(columns=rel_df_di.columns.values.tolist()\
                              +vis_list)
rel_df_di[vis_list] = rel_dfk.apply(lambda row: pd.Series(decomposeCArray(\
    degVis(ant_sep, makeCArray(row[len(min_list):len(min_list) + \
    md['no_unq_bls']*2].values.astype(float)), *row[-3:].values.astype(float)))), \
    axis=1)

# merging dataframes
rel_df_di['JD'] = jd_ci
rel_df_di.reset_index(inplace=True)
rel_df_di.set_index(['freq', 'time_int', 'JD'], inplace=True)
rel_df_di.sort_index(inplace=True)

rel_df_t = pd.concat([rel_df_t, rel_df_di])
rel_df_t.sort_index(inplace=True)

In [ ]:
rel_df_t